In [ ]:
from itertools import product, combinations, count, permutations
import numpy as np
import pandas as pd
from astroquery.vizier import Vizier
import matplotlib.pyplot as plt
import astropy as ap
import sklearn
from sklearn.neighbors import KernelDensity as KD
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA, KernelPCA
import corner
from sklearn.mixture import GaussianMixture as GM
from sklearn.mixture import BayesianGaussianMixture as BGM
from scipy.stats import norm
from matplotlib.colors import LogNorm
from astroquery.gaia import Gaia
from scipy import stats

In [ ]:
%matplotlib inline

In [ ]:
Vizier = Vizier(row_limit=20000)

In [ ]:
catalog = Vizier.get_catalogs("J/A+A/618/A93")

clucata = catalog[1]

In [ ]:
newc = clucata.group_by('Cluster')
maxcluster = np.argmax(newc.groups.indices[1:]-newc.groups.indices[:-1])
clusterid = maxcluster+6
bigcluster = newc.groups[clusterid]
print(bigcluster.colnames)
newc.groups[clusterid]['Cluster'][0]

Good `'Cluster'`s to choose from: Alessi\_24, ASCC_99, Alessi\_12

In [ ]:
cutcluster = bigcluster[bigcluster['PMemb']>.8]
cutcluster = cutcluster[~np.isnan(cutcluster["BP-RP"])]
#We should cut by lines away from main sequence
cutcluster = cutcluster[~np.logical_and(cutcluster["BP-RP"]>1.0,cutcluster["Gmag"]<10.)]
cutcluster = cutcluster[~np.logical_and(cutcluster["BP-RP"]<.7, cutcluster["Gmag"]>13.8)]
plt.plot(cutcluster['RA_ICRS'],cutcluster['DE_ICRS'],'+')
plt.title('angular coordinates of '+cutcluster['Cluster'][0])

In [ ]:
query = 'SELECT source_id, phot_bp_mean_mag, phot_rp_mean_mag FROM gaiadr2.gaia_source WHERE source_id = {}'

In [ ]:
newquery = query.format(' OR source_id = '.join(list(map(str, cutcluster['Source']))))
newquery;

In [ ]:
Gaia.launch_job('SELECT source_id, phot_bp_mean_mag, phot_rp_mean_mag FROM gaiadr2.gaia_source WHERE source_id = 1842846157478382720')

In [ ]:
newjob = Gaia.launch_job(query=newquery)

In [ ]:
results = newjob.get_results()

In [ ]:
results['phot_bp_mean_mag'].info.parent_table;

In [ ]:
cutcluster.add_columns([results['phot_bp_mean_mag'],results['phot_rp_mean_mag']])

In [ ]:
fig = plt.figure(figsize=(20,20))
for a,i in zip(permutations(['phot_bp_mean_mag','Gmag','phot_rp_mean_mag', 'BP-RP'], 2), count(1)):
    print(a)
    print(stats.linregress(cutcluster[a[0]],cutcluster[a[1]]))
    ax = fig.add_subplot(4,3, i)
    ax.scatter(cutcluster[a[0]],cutcluster[a[1]])
    ax.set_xlabel(a[0])
    ax.set_ylabel(a[1])

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter3D(cutcluster['Gmag'],cutcluster['phot_bp_mean_mag'],cutcluster['phot_rp_mean_mag']);

In [ ]:
plt.hist(cutcluster['PMemb'])
plt.xlabel('PMemb')
plt.title('cluster membership probability of '+cutcluster['Cluster'][0])

In [ ]:
plt.plot(cutcluster['BP-RP'],cutcluster['Gmag'], '+')
plt.ylim(19, 7)
plt.xlabel('BP-RP')
plt.ylabel('Gmag')
plt.title('color-magnitude diagram of '+cutcluster['Cluster'][0])

PCA - diagram, inverse of variance as metric -> sqrt(thing)= transformation, apply ->PCA(test) -> KDE -> inverse transform both L and R on Kernel Widths

LLE? -> Local linear embedding -> for non MS

remove outliers before PCA

In [ ]:
X = np.asarray(np.vstack((cutcluster['BP-RP'], cutcluster['Gmag']))).T

In [ ]:
pca = PCA(n_components=2)
X_pca = pca.fit(X)

In [ ]:
X_transform = pca.transform(X)
X_transform.shape
plt.scatter(X_transform[:,0], X_transform[:,1])
plt.axis("equal")

In [ ]:
np.sqrt(pca.singular_values_)

In [ ]:
def draw_vector(v0, v1, ax=None):
    ax = ax or plt.gca()
    arrowprops=dict(arrowstyle='->',
                    linewidth=2,
                    shrinkA=0, shrinkB=0)
    ax.annotate('', v1, v0, arrowprops=arrowprops)

# plot data
plt.scatter(X[:, 0], X[:, 1], alpha=0.2)
for length, vector in zip(pca.explained_variance_, pca.components_):
    v = vector * 3 * np.sqrt(length)
    draw_vector(pca.mean_, pca.mean_ + v)
plt.axis('equal');
plt.ylim(18,7)
plt.xlim(-2,4)

In [ ]:
X_pca = pca.transform(X)
#plt.scatter(X_pca[:, 0], X_pca[:, 1])
plt.axis('equal')
X_pca[:,0] = X_pca[:,0]/np.sqrt(pca.singular_values_[0])
X_pca[:,1] = X_pca[:,1]/np.sqrt(pca.singular_values_[1])
plt.scatter(X_pca[:, 0], X_pca[:, 1]);

In [ ]:
# I couldn't immediately find a KDE code that enabled different bandwidth in each dimension
# which we want because the errors in color are much greater than the errors in magnitude
params = {'bandwidth': np.logspace(-2, 2, 200)}
grid = GridSearchCV(KD(kernel='gaussian'), params, cv=10)
grid.fit(X_pca)

print("best bandwidth: {0}".format(grid.best_estimator_.bandwidth))
# first attempt obviously too fine a bandwidth because it allows for double stars
# we could fix it here or say this is just what the data is and fit an HRD model that doesn't permit those
# so now the data is the KDE evaluated on a grid

xmin = -1
xmax = +2
ymin = -1
ymax = +1
xlen = 50
ylen = 50


kde = grid.best_estimator_.fit(X_pca)
eval_where = np.array(list(product(np.linspace(xmin,xmax,xlen), np.linspace(ymin,ymax, ylen))))
log_dens = kde.score_samples(eval_where)

plt.imshow(np.flip(np.exp(log_dens.reshape(xlen, ylen).T), axis=0),
           extent=[xmin, xmax, ymin, ymax])
#plt.scatter(cutcluster['BP-RP'], cutcluster['Gmag'], marker='.', color='r', s=1)

In [ ]:
a = np.linspace(xmin,xmax,xlen)*np.sqrt(pca.singular_values_[0])
b = np.linspace(ymin,ymax,ylen)*np.sqrt(pca.singular_values_[1])
A, B = np.meshgrid(a, b)
print(B.shape)
plt.contour(A, B, np.exp(log_dens.reshape(xlen,ylen).T));

In [ ]:
C_grid = np.array([np.ravel(A), np.ravel(B)]).T
E_grid = pca.inverse_transform(C_grid)[:,0].reshape(xlen, ylen)
F_grid = pca.inverse_transform(C_grid)[:,1].reshape(xlen, ylen)

In [ ]:
plt.plot(cutcluster['BP-RP'],cutcluster['Gmag'], 'r+',alpha=.4)
plt.contour(E_grid, F_grid, np.exp(log_dens.reshape(xlen,ylen).T))
plt.ylim(19,8)
plt.xlabel('BP-RP')
plt.ylabel('Gmag')
plt.title('color-magnitude diagram of '+cutcluster['Cluster'][0])
plt.savefig("cmd.png")

In [ ]:
samps = np.array([[x*np.sqrt(pca.singular_values_[0]),y*np.sqrt(pca.singular_values_[1])] for x,y in kde.sample(100)])

In [ ]:
cmsamps = pca.inverse_transform(samps)

In [ ]:
mags = cmsamps[:,1]
colors = cmsamps[:,0]

In [ ]:
magsum = lambda x: -5./2*np.log10(np.sum(10.**(-2./5.*x)))

In [ ]:
makepairwise = lambda x: np.array(list(product(x, repeat=2)))

In [ ]:
newmags = np.array(list(map(magsum, makepairwise(mags))))

### END OF REAL WORK

In [ ]:
plt.hist(X_pca[:,1], bins=20);

plt.imshow(np.exp(pred.reshape(100, 100).T), extent=[0., 2.7, 18., 7.], aspect=0.25)

In [ ]:
gmm = BGM(n_components=4, n_init=30, tol=1e-8,max_iter=2000)
gmm.fit(X_pca[:,1].reshape(-1,1))

In [ ]:
gaussians = [norm(μ, np.sqrt(σ[0])) for μ, σ in zip(gmm.means_, gmm.covariances_)]

In [ ]:
X_test, label = gmm.sample(X_pca[:,1].shape[0])
X_space = np.linspace(-1, 4, 1000)
for gauss, weight in zip(gaussians, gmm.weights_):
    plt.plot(X_space, gauss.pdf(X_space)*weight)
plt.hist([X_test[:,0], X_pca[:,1]], bins=30, density=True);

In [ ]:
labels = gmm.predict(X_pca[:,1].reshape(-1,1))
scores = gmm.predict_proba(X_pca[:,1].reshape(-1,1))[:,0]

In [ ]:
pca.inverse_transform([[-4,-1],[-4,1],[6,-1],[6,1]])

In [ ]:
X_new = pca.inverse_transform(X_pca)
plt.scatter(X_new[:, 0], X_new[:, 1], color='g', alpha=0.4)
plt.imshow(np.flip(np.exp(log_dens.reshape(50, 50).T), axis=0),extent=[])
plt.ylim(18,7)

In [ ]:
corner.corner(X_transform);

In [ ]:
plt.scatter(X_back[:, 0], X_back[:, 1], c="red",
            s=20, edgecolor='k')

In [ ]:
cmd = np.asarray(np.vstack((cutcluster['BP-RP'], cutcluster['Gmag']))).T
print((np.min(cutcluster['BP-RP']), np.max(cutcluster['BP-RP']), np.min(cutcluster['Gmag']), np.max(cutcluster['Gmag'])))

In [ ]:
cmd.shape

In [ ]:
x = np.array([np.ravel(E_grid), np.ravel(F_grid)]).T
y = log_dens

In [ ]:
kernel = 5*kernels.Matern52Kernel(5., ndim=2)
gp = george.GP(kernel, mean=np.mean(y), fit_mean=True)

In [ ]:
gp.compute(x)

test_x = np.array(list(product(np.linspace(0, 3, 25),np.linspace(7,18,25))))

In [ ]:
pred, pred_var = gp.predict(y, test_x)

In [ ]:
plt.imshow(np.flip(np.exp(pred.reshape(100, 100).T),axis=0), extent=[0., 3, 18., 7.], aspect=0.25)

In [ ]:
import scipy.optimize as op

# Define the objective function (negative log-likelihood in this case).
def nll(p):
    gp.set_parameter_vector(p)
    ll = gp.log_likelihood(y, quiet=True)
    return -ll if np.isfinite(ll) else 1e25

# And the gradient of the objective function.
def grad_nll(p):
    gp.set_parameter_vector(p)
    return -gp.grad_log_likelihood(y, quiet=True)

# You need to compute the GP once before starting the optimization.
gp.compute(x)

# Print the initial ln-likelihood.
print(gp.log_likelihood(y))

# Run the optimization routine.
p0 = gp.get_parameter_vector()
results = op.minimize(nll, p0, jac=grad_nll, method="L-BFGS-B")

# Update the kernel and print the final log-likelihood.
gp.set_parameter_vector(results.x)
print(gp.log_likelihood(y))



In [ ]:
pred, pred_var = gp.predict(y, test_x)


plt.scatter(cutcluster['BP-RP'], cutcluster['Gmag'], marker='.', color='r', s=1)
plt.imshow(np.exp(pred.reshape(100, 100).T), extent=[0., 3, 18., 7.], aspect=0.25)

In [ ]:
n_components = 5

In [ ]:
gmm2 = BGM(n_components=n_components, n_init=50, tol=1e-8)
gmm2.fit(X)
labels2 = gmm2.predict(X);

In [ ]:
ℵ = np.linspace(0., 5, 100)
ℶ = np.linspace(8., 18., 100)
ℷ, ℸ = np.meshgrid(ℵ, ℶ)
ΓΓ = np.array([ℷ.ravel(), ℸ.ravel()]).T
Ξ = -gmm2.score_samples(ΓΓ)
Ξ = Ξ.reshape(ℷ.shape)

plt.contour(ℷ, ℸ, Ξ, norm=LogNorm(vmin=1.0, vmax=1000.0),
                 levels=np.logspace(0, 3, 10))
for i in range(n_components):
    plt.plot(X[:,0][labels2==i], X[:,1][labels2==i], '+')

plt.ylim(19,7)
plt.xlim(0,2)

In [ ]:
X1, X2 = np.meshgrid(np.linspace(0, 5, 50), np.linspace(18, 7, 50))
X_grid = np.array([np.ravel(X1), np.ravel(X2)]).T


In [ ]:
sampcmd, junk̅ = gmm2.sample(300)
plt.plot(sampcmd[:,0], sampcmd[:,1], '+')
plt.ylim(18,7)